# 从网页中截取文章内容

## 步骤1：参数设置

In [1]:
url = 'http://www.infoq.com/cn/articles/introduction-of-tensorflow-part4'
print(url)

http://www.infoq.com/cn/articles/introduction-of-tensorflow-part4


## 步骤2：查找相应的截取关键字

In [2]:
if 'blog.jobbole.com' in url:  # 伯乐在线
    title_key = '.entry-header'
    content_key = '.entry'
    name = "伯乐在线" 
#     print('伯乐在线 http://blog.jobbole.com')
elif 'blog.csdn.net' in url:  # csdn
    title_key = '.link_title'
    content_key = '#article_content'
    name = "csdn"
#     print('csdn http://blog.csdn.net')
elif 'www.codingpy.com' in url:  # 编程派网址
    title_key = '.header h1'
    content_key = '.article-content'
    name = "编程派网址"
#     print('编程派网址 http://www.codingpy.com')
elif 'www.infoq.com' in url:  # InfoQ
    title_key = '.title_canvas'
    content_key = '.text_info_article'
    name = "InfoQ"
#     print('InfoQ：' + url)
else:
    title_key = ''
    content_key = ''
    name = "其他"
print(name)
print("title_key: " + title_key)
print("content_key: " + content_key)

InfoQ
title_key: .title_canvas
content_key: .text_info_article


## 步骤3：发出网页请求，接收响应

In [3]:
from urllib import request
req = request.Request(url)
res = request.urlopen(req)
html = res.read().decode('utf-8')
print(html)

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">










<html xmlns="http://www.w3.org/1999/xhtml">
    <head>
        <title>深入浅出Tensorflow（四）：卷积神经网络</title>
        <link rel="canonical" href="http://www.infoq.com/cn/articles/introduction-of-tensorflow-part4"/>
        
        <meta http-equiv="pragma" content="no-cache" />
<meta http-equiv="cache-control" content="no-cache" />
<meta http-equiv="expires" content="0" />
<meta http-equiv="content-type" content="text/html; charset=utf-8" />
<meta name="copyright" content="&copy; 2006 C4Media" />

<meta name="google-site-verification" content="0qInQx_1WYOeIIbxnh7DnXlw1XOxNgAYakO2k4GhNnY" />
<meta name="msapplication-TileColor" content="#ffffff"/>
<meta name="msapplication-TileImage" content="/styles/i/logo.jpg"/>
<meta property="wb:webmaster" content="3eac1729a8bbe046" />

        
		
			<link href='http://q.infoqstatic.com/static/css/index.css' rel='stylesheet'

## 步骤4：从网页中提取文章标题和内容

In [4]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')
title = soup.select(title_key)[0].text.strip()  # 文章标题
print('文章标题：', title)
content = soup.select(content_key)[0]  # 文章内容
# content

文章标题： 深入浅出Tensorflow（四）：卷积神经网络


## 步骤5：下载文章中的图片

### 5.1 哈希码生成函数，用于给图片重新命名

In [5]:
import hashlib
def md5(name):
    """ 将字符串转成哈希码 """
    if not isinstance(name, str):
        name = str(name)
    md5 = hashlib.md5()
    md5.update(name.encode('utf-8'))
    return md5.hexdigest()

### 5.2 下载图片，并修改文章图片的超链接

In [6]:
import re
import os
content = str(content)
pattern = '<img .*?src=\"(.*?)\"'
re_image = re.compile(pattern)
for image_link in re_image.findall(content):
    if not os.path.exists('images'):
        os.mkdir('images')
    filename = 'images/' + md5(image_link) + os.path.splitext(image_link)[-1]
    try:
        request.urlretrieve(image_link, filename)
        print('下载完成', filename)
    except Exception as e:
        print('图片出错', e)
    else:
        content = content.replace(image_link, filename)
print('已完成--------')

下载完成 images/6885005bf72a9089046b13d551fd6d4e.jpg
下载完成 images/c702d14c292ae7dded7103c05b998131.jpg
下载完成 images/ac5e1b19fa58338a6750f1f64b6406a8.jpg
下载完成 images/99f1064bcfd3dd598adc7759638ca6fb.jpg
下载完成 images/85cf5e072f63c4925e3cdaaf410ca39c.jpg
下载完成 images/5858aeb008e7c865e796c5ee7c7a044e.jpg
下载完成 images/f42b1664b50831e65642c52af26afbbe.jpg
下载完成 images/d26f49b8718edd35d98b70dbd50f8979.jpg
下载完成 images/b7d591b4c0a38181db770da4cc085601.jpg
已完成--------


## 步骤6：将截取的文章标题和内容重新组合成新的网页文件

In [7]:
html_template = """<!DOCTYPE html>
<html><head><meta charset="UTF-8">
</head><body>
<p><a href="{origin}">原文链接</a></p>
<p><center><h1>{title}</h1></center></p>
    {content}
</body></html>"""
html = html_template.format(origin=url, title=title, content=content)
print(html)

<!DOCTYPE html>
<html><head><meta charset="UTF-8">
</head><body>
<p><a href="http://www.infoq.com/cn/articles/introduction-of-tensorflow-part4">原文链接</a></p>
<p><center><h1>深入浅出Tensorflow（四）：卷积神经网络</h1></center></p>
    <div class="text_info text_info_article">
<p>2017年2月16日，Google正式对外发布Google TensorFlow 1.0版本，并保证本次的发布版本API接口完全满足生产环境稳定性要求。这是TensorFlow的一个重要里程碑，标志着它可以正式在生产环境放心使用。在国内，从InfoQ的判断来看，TensorFlow仍处于创新传播曲线的创新者使用阶段，大部分人对于TensorFlow还缺乏了解，社区也缺少帮助落地和使用的中文资料。InfoQ期望通过深入浅出TensorFlow系列文章能够推动Tensorflow在国内的发展。欢迎加入QQ群（群号：183248479）深入讨论和交流。下面为本系列的前三篇文章：</p>
<p><a href="http://www.infoq.com/cn/articles/introduction-of-tensorflow-part01">深入浅出Tensorflow（一）：深度学习及TensorFlow简介</a></p>
<p><a href="http://www.infoq.com/cn/articles/introduction-of-tensorflow-part02">深入浅出TensorFlow（二）：TensorFlow解决MNIST问题入门 </a></p>
<p><a href="http://www.infoq.com/cn/articles/introduction-of-tensorflow-part03">深入浅出Tensorflow（三）：训练神经网络模型的常用方法</a></p>
<p><strong><span font-size:="" hiragino="" micro="" microsoft="" open

## 步骤7：将文件写入磁盘

In [8]:
import codecs
filename = title + ".html"
with codecs.open(filename, "w", "utf-8") as f:
    f.write(html.replace(u'\xa0', u' ').replace(u'\U0001f60a', u' '))  # 在windows中出错，所以这里进行了字符串替换

## (完)